In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from deform_rl.envs.Rectangle_env.environment import Rectangle1D
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import EvalCallback

import pygame

In [3]:
def make_env(rank,seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = Rectangle1D(render_mode='human',oneD=False)
        env = TimeLimit(env,max_episode_steps=1000)
        env = Monitor(env)
        # use a seed for reproducibility
        # Important: use a different seed for each environment
        # otherwise they would generate the same experiences
        env.reset(seed=seed + rank)
        return env

    set_random_seed(seed)
    return _init



env0 = DummyVecEnv([make_env(i+4) for i in range(4)])
training_env =VecNormalize(env0)
# training_env = env0
env1 = DummyVecEnv([make_env(i+4) for i in range(1)])
validation_env = VecNormalize(env1)

save_dir = os.path.join("saved_models")
log_dir = os.path.join("logs")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)



In [32]:
import torch.nn as nn
from stable_baselines3.common.policies import ActorCriticPolicy

class SeparateLinearPolicy(ActorCriticPolicy):
    def __init__(self, *args, **kwargs):
        super(SeparateLinearPolicy, self).__init__(*args, **kwargs,
                                                   squash_output=False)
        self.policy_net = nn.Sequential(
            nn.Linear(self.observation_space.shape[0], self.action_space.shape[0],bias=True)
        )
        self.value_net = nn.Sequential(
            nn.Linear(self.observation_space.shape[0], 1,bias=True)
        )

    def forward(self, obs, deterministic = True):
        return self.policy_net(obs), self.value_net(obs)
    
    def forward_actor(self, obs, deterministic = True):
        return self.policy_net(obs)
    def forward_critic(self, obs, deterministic = True):
        return self.value_net(obs)

In [4]:
# policy_kwargs = dict(activation_fn=nn.Identity,net_arch=dict(pi=[],vf=[]))
model = PPO('MlpPolicy', training_env,device='cpu',
            verbose=1,tensorboard_log=log_dir)
model.learn(500000,tb_log_name='rectangle_v2') 
model.save("shit_model")
training_env.save("shitNormalize.pkl")


Using cpu device
Logging to logs/rectangle_v2_14
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 912       |
|    ep_rew_mean     | -6.36e+03 |
| time/              |           |
|    fps             | 4887      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 8192      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 956          |
|    ep_rew_mean          | -5.46e+03    |
| time/                   |              |
|    fps                  | 2586         |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0054368703 |
|    clip_fraction        | 0.056        |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |

In [17]:
model.learn(300000,tb_log_name='rectangle_v2',reset_num_timesteps=False) 


Logging to logs/rectangle_v2_10
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 574       |
|    ep_rew_mean     | -1.29e+03 |
| time/              |           |
|    fps             | 1160      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 804864    |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 575          |
|    ep_rew_mean          | -1.3e+03     |
| time/                   |              |
|    fps                  | 825          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 806912       |
| train/                  |              |
|    approx_kl            | 0.011744081  |
|    clip_fraction        | 0.117        |
|    clip_range           | 0.2          |
|    entropy_loss         | -3.3         |
|    explained_v

KeyboardInterrupt: 

In [4]:
# model.learn(800000,tb_log_name='rectangle_v2',reset_num_timesteps=False) 
# model.save("shit_model")
# training_env.save("shitNormalize.pkl")


In [5]:
env2 = DummyVecEnv([make_env(i+4) for i in range(1)])
testing_env = VecNormalize.load('shitNormalize.pkl',env2)

testing_env.training = False
# testing_env = env2
model = PPO.load('shit_model',testing_env)
print(model.policy)

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=2, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)


In [8]:
obs = testing_env.reset()
cum_reward = cnt=0
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    
    obs, reward, done, info = testing_env.step(action)
    cum_reward += reward
    cnt +=1
    if done:
        obs = testing_env.reset()
        print("Episode done: ", cnt, "reward: ", cum_reward)
        
        cnt=0
        cum_reward=0
    testing_env.render()
    if pygame.event.get(pygame.QUIT):
        break
    # print(obs, reward)
testing_env.close()

Episode done:  43 reward:  [1.25827794]
Episode done:  86 reward:  [0.84548233]
Episode done:  69 reward:  [0.73622749]
Episode done:  102 reward:  [0.9106718]
Episode done:  42 reward:  [1.36424045]
Episode done:  62 reward:  [1.23878087]
Episode done:  110 reward:  [0.84254603]


Episode done:  95 reward:  [214.71915]
Episode done:  74 reward:  [271.5203]
Episode done:  65 reward:  [299.72574]
Episode done:  73 reward:  [274.79398]
Episode done:  59 reward:  [316.1392]
Episode done:  67 reward:  [293.01675]
Episode done:  77 reward:  [264.3034]
Episode done:  58 reward:  [318.39868]
Episode done:  113 reward:  [164.8003]
Episode done:  51 reward:  [338.93597]
Episode done:  45 reward:  [356.6471]
Episode done:  85 reward:  [242.5853]
Episode done:  87 reward:  [238.1221]